In [1]:
import json
import pprint as pp
import pandas as pd
import tqdm

In [5]:
with open("persons_parsed.json", 'r') as f:
    persons = json.load(f)

In [51]:
pp.pprint(persons["583"])

{'family_name': 'Прохоров',
 'given_name': 'Михаил',
 'id': 583,
 'name': 'Прохоров Михаил Дмитриевич',
 'patronymic_name': 'Дмитриевич',
 'sections': [{'bonds': ['15 759 996 995,0 руб. 1 шт. право требования по '
                         'договорам займа и договору купли-продажи',
                         '6 044 239 929,0 руб. 28 шт. вексель'],
               'incomes': [{'comment': 'зарплата, «ГМК «Норильский Никель»,\r\n'
                                       'зарплата, ООО «Группа ОНЭКСИМ», '
                                       'вознаграждение, ОАО «Международный '
                                       'аэропорт Шереметьево», \r\n'
                                       'авторское вознаграждение, ООО «Русский '
                                       'пионер», \r\n'
                                       'доходы по операциям с ценными бумагами '
                                       'и другими финансовыми '
                                       'инструментами, \r\n'
         

In [47]:
def family_comp(pid, persons):
    person = persons[pid]
    res = {
        "name": person["name"],
        "id": person["id"],
        "years": {}
    }
    
    for declaration in person["sections"]:
        year = declaration["main"]["year"]
        res["years"][year] = {"personal_income": 0, "family_income": 0, "childrens_income": 0,\
                                                    "personal_sq": 0, "family_sq": 0, "childrens_sq": 0,\
                                                    "personal_veh": 0, "family_veh": 0, "childrens_veh": 0}
        for income in declaration["incomes"]:
            if income["relative"] == None:
                res["years"][year]["personal_income"] += float(income["size"]) if float(income["size"]) != None else 0
            elif income["relative"]["id"] == 2:
                res["years"][year]["family_income"] += float(income["size"]) if float(income["size"]) != None else 0
            else:
                res["years"][year]["childrens_income"] += float(income["size"]) if float(income["size"]) != None else 0
                
                
        for estate in declaration["real_estates"]:
            if estate["relative"] == None:
                res["years"][year]["personal_sq"] += estate["square"] if estate["square"] != None else 0
            elif estate["relative"]["id"] == 2:
                res["years"][year]["family_sq"] += estate["square"] if estate["square"] != None else 0
            else:
                res["years"][year]["childrens_sq"] += estate["square"] if estate["square"] != None else 0
                
        for veh in declaration["vehicles"]:
            if veh["relative"] == None:
                res["years"][year]["personal_veh"] += 1
            elif veh["relative"]["id"] == 2:
                res["years"][year]["family_veh"] += 1
            else:
                res["years"][year]["childrens_veh"] += 1
                
        res["years"][year]["total_income"] = res["years"][year]["personal_income"] \
                                            + res["years"][year]["childrens_income"] \
                                            + res["years"][year]["family_income"]
        
        res["years"][year]["total_sq"] = res["years"][year]["personal_sq"] \
                                            + res["years"][year]["childrens_sq"] \
                                            + res["years"][year]["family_sq"]
        
        res["years"][year]["total_veh"] = res["years"][year]["personal_veh"] \
                                            + res["years"][year]["childrens_veh"] \
                                            + res["years"][year]["family_veh"]
    return res

In [48]:
family_results = {}

for key in tqdm.tqdm(persons):
    family_results[key] = family_comp(key, persons)

100%|██████████| 132784/132784 [00:04<00:00, 28906.68it/s]


In [49]:
fields = ["personal_income", "childrens_income", "family_income", "personal_sq", "childrens_sq",\
         "family_sq", "personal_veh", "childrens_veh", "family_veh"]

for field in fields:
    max_val = 0
    max_id = 0
    max_year = 0
    for pid in family_results:
        person = family_results[pid]
        for year in person["years"]:
            if person["years"][year][field] > max_val:
                max_val = person["years"][year][field]
                max_id = person["id"]
                max_year = year
              
    print("max " + field + " is " + str(max_val) + " belongs to " + persons[str(max_id)]["name"] + "  " + str(max_id) + " " + str(max_year)) 

max personal_income is 115253119846.38 belongs to Прохоров Михаил Дмитриевич  583 2010
max childrens_income is 73333333.0 belongs to Конюшков Алексей Алексеевич  22158 2017
max family_income is 2350680824.66 belongs to Минниханов Рустам Нургалиевич  423 2016
max personal_sq is 22971003264.600002 belongs to Крюк Василий Николаевич  15335 2015
max childrens_sq is 186875074.3 belongs to Захлевная Т. А.  48239 2015
max family_sq is 1741863843.0999997 belongs to Терехова Светлана Алексеевна  14576 2015
max personal_veh is 205 belongs to Палкин Андрей Васильевич  12196 2015
max childrens_veh is 4 belongs to Зюганов Геннадий Андреевич  8 1999
max family_veh is 128 belongs to Богомаз Александр Васильевич  1135 2018
